In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 10.1 MB/s 
     |████████████████████████████████| 6.5 MB 51.3 MB/s 
     |████████████████████████████████| 895 kB 74.0 MB/s 
     |████████████████████████████████| 77 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import random
import scipy.sparse as sp
import torch
import os
import time
import pandas as pd
import pickle
import torch.nn.functional as F
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from transformers import GPT2Tokenizer, GPT2Model
import nltk
from keras_preprocessing.text import Tokenizer
from torch import nn
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve 
from sklearn.metrics import roc_auc_score
from scipy.sparse import coo_matrix, vstack

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # TF 2.1
random.seed(SEED)
#seed(SEED)
np.random.seed(SEED)

In [4]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# BOW

In [ ]:
def sentence_process(sentence):
  comment = sentence.replace('\n', ' ')
  while '\"' in comment :
    comment = comment.replace('\"', '')
  while "\'" in comment :
    comment = comment.replace("\'", '')
  while ':' in comment :
    comment = comment.replace(':', '')
  while '.' in comment :
    comment = comment.replace('.', '')
  while '@' in comment :
    comment = comment.replace('@', '')
  while '+' in comment :
    comment = comment.replace('+', '')
  while '=' in comment:
    comment = comment.replace('=', '')
  while '&' in comment :
    comment = comment.replace('&', '')
  while ')' in comment or '(' in comment:
    comment = comment.replace(')', '').replace('(', '')
  while ',' in comment or ':' in comment or ';' in comment:
    comment = comment.replace(":", '').replace(',', '').replace(';', '')
  for c in comment :
    if c.isascii() == False :
      comment = comment.replace(c, '')
  n1 = comment.count('!')
  n2 = comment.count('?')
  for i in range(n1):
    comment = comment.replace('!', ' !')
  for i in range(n2):
    comment = comment.replace('?', ' ?')
  while '  ' in comment :
    comment = comment.replace('  ', ' ')
  return comment

def bow(sentence,dic):
  v=np.zeros(len(dic))
  for word in dic :
    words = sentence.split(' ')
    for w in words :
      if word == w:
        v[dic[word]]+=1
  return(coo_matrix(v))

class BOWDataset :
    def __init__(self, filepath, n_words):
        self.filepath = filepath
        self.n_words = n_words

    def process(self):
        tic = time.time()

        train_df = pd.read_csv(self.filepath + 'train.csv')
        tic = time.time()

        #delete comments that belongs to several classes
        idx_to_del = []
        for row in train_df.itertuples():
          if sum(row[3::])> 1 :
            idx_to_del.append(row[0])
          
        train_df = train_df.drop(idx_to_del)

        cpt = 0
        n_too_long = 0
        toxic = train_df[train_df['toxic'] == 0]
        obscene = toxic[toxic['obscene']==0]
        threat = obscene[obscene['threat']==0]
        insult = threat[threat['insult']==0]
        neutral = insult[insult['identity_hate']==0] #df with neutral comment
        toxic = train_df[train_df['toxic'] == 1]
        obscene = train_df[train_df['obscene'] == 1]
        threat = train_df[train_df['threat'] == 1]
        insult = train_df[train_df['insult'] == 1]
        identity_hate = train_df[train_df['identity_hate'] == 1]
        final_df = pd.concat([toxic, obscene, threat, insult, identity_hate, neutral]) #merge all the df together in the right order
        
        dic = {}
        n_dic=0
        if False :
          for row in final_df.itertuples():
            
            cpt = cpt + 1
            if cpt in [1344, 1345, 1445, 1446, 1528, 1529, 3243, 3244, 3773, 3776, 11888] :
                continue

            if n_dic>=10000:
                break
            print("cpt dic:", cpt)
            
            comment = sentence_process(row[2])
            words = comment.split(' ')
            while '' in words :
              words.remove('')
            for w in words:
              if w.isnumeric() and len(w)>=5:
                words.remove(w)
            words = list(filter(lambda s: 'http' not in s, words))
            comment = ' '.join(words)

            if len(words)>100 : 
              print("Too long")
              n_too_long += 1
              continue

            n_dic=n_dic+1
            for word in words:
              if word not in dic:
                dic[word]=len(dic)

        #with open(self.filepath +'dic.pickle', 'wb') as handle:
          #pickle.dump(dic, handle)

        with open(self.filepath +'dic.pickle', 'rb') as handle:
          dic = pickle.load(handle)

        if True:
          cpt = 0
          X = []
          self.labels = []
          for row in final_df.itertuples():
        
            cpt = cpt + 1
            print("cpt X", cpt)

            if cpt in [1344, 1345, 1445, 1446, 1528, 1529, 3243, 3244, 3773, 3776, 11888] :
              continue

            if np.shape(X)[0] == self.n_words and np.shape(self.labels[:self.n_words])[0] == self.n_words:
              break
            
            comment = sentence_process(row[2])
            words = comment.split(' ')
            while '' in words :
              words.remove('')
            for w in words:
              if w.isnumeric() and len(w)>=5:
                words.remove(w)
            words = list(filter(lambda s: 'http' not in s, words))
            comment = ' '.join(words)

            if len(words)>100 : 
              print("Too long")
              n_too_long += 1
              continue

            if row[3] == 0 and row[4] == 0 and row[5] ==0 and row[6] ==0 and row[7] == 0 and row[8] == 0:
              self.labels.append(0)
            elif row[3] == 1 :
              self.labels.append(1)
            elif row[4] == 1 :
              self.labels.append(6)
            elif row[5] == 1 :
              self.labels.append(2)
            elif row[6] == 1 :
              self.labels.append(3)
            elif row[7] == 1 :
              self.labels.append(4)
            else: 
              self.labels.append(5)

            X.append(bow(comment,dic))

          X = vstack(X)
          labels = np.array(self.labels)
          print("labels", np.unique(labels))
        with open('/content/drive/My Drive/NLP/X_10000.pickle', 'wb') as handle:
          pickle.dump(X, handle)

        with open('/content/drive/My Drive/NLP/labels_10000.pickle', 'wb') as handle: 
          pickle.dump(labels, handle)
        #with open('/content/drive/My Drive/NLP/X_10000.pickle', 'rb') as handle:
          #X=pickle.load(handle)

        #with open('/content/drive/My Drive/NLP/labels_10000.pickle', 'rb') as handle: 
          #labels=pickle.load(handle)
        return X, labels

X, labels = BOWDataset(filepath = '/content/drive/My Drive/NLP/', n_words = 10000).process()

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

random_states = [0,1,2]

accuracy = []
F1_Score=[]
AUC=[]

tic = time.time()

#class_weights = compute_class_weight(class_weight='balanced',classes=np.unique(labels),y=labels)
#print(class_weights)

for r in random_states :

  print("Random state : ", r, "\n")
  X, labels = shuffle(X, labels, random_state=r)
  num_examples = np.shape(X)[0]
  num_train = int(0.80 * num_examples)
  num_val = int((num_examples-num_train)/2)
  X_train, y_train = X[:num_train], labels[:num_train]
  X_test, y_test = X[(num_train+num_val) : (num_train+2*num_val)], labels[(num_train+num_val):(num_train+2*num_val)]
  #X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.80,random_state=r)
  pipeline = Pipeline([('clf', LogisticRegression(random_state = r,class_weight="balanced"))])

  params = {'clf__C': [1.0,0.5,0.1], 'clf__max_iter':[500,1000]}


  rskf = StratifiedKFold(n_splits=6, random_state=r, shuffle=True)

  cv = GridSearchCV(pipeline, params, cv = rskf, scoring = 'accuracy')
  cv.fit(X_train, y_train)

  #print(f'Best accuracy -score: {cv.best_score_:.3f}\n')
  #print(f'Best parameter set: {cv.best_params_}\n')
  #print(f'Scores: {classification_report(y_train, cv.predict(X_train))}')

  preds_proba = cv.predict_proba(X_test)
  #print(preds_proba)
  preds = cv.predict(X_test)
  #print(f'Scores: {classification_report(y_test, preds)}\n')
  #print(f'accuracy score: {accuracy_score(y_test, preds):.3f}')
  accuracy.append(accuracy_score(y_test, preds))
  F1_Score.append(f1_score(y_test, preds, average='weighted'))
  AUC.append(roc_auc_score(y_test,preds_proba,multi_class="ovo",average="weighted"))

tac = time.time()

print("Accuracy : ",np.mean(accuracy),"with std",np.std(accuracy))
print("f1_score : ",np.mean(F1_Score),"with std",np.std(F1_Score))
print("AUC : ",np.mean(AUC),"with std",np.std(AUC))
print("time : ", tac-tic)

Random state :  0 

Random state :  1 

Random state :  2 

Accuracy :  0.7566666666666667 with std 0.004784233364802446
f1_score :  0.7531432264466863 with std 0.006082794528862612
AUC :  0.7406805422311146 with std 0.034569295289029724
time :  828.293196439743
